In [1]:
import json
from pprint import pprint
import h5py
import scipy.io
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

Using TensorFlow backend.


In [2]:
with open('sg_dataset/sg_test_annotations.json') as f:
    data = json.load(f)
#Extracting the required data
data[4].get('objects','')[0].get('bbox','')

{'y': 148.0, 'x': 0.0, 'w': 286.0, 'h': 282.0}

In [3]:
data[0].get('filename','')

'3845770407_1a8cd41230_b.jpg'

In [4]:
def get_image(data_in):
    img = data[data_in].get('filename','')
    DATADIR = "sg_dataset"
    CATEGORIES = ["sg_test_images"]
    
    for category in CATEGORIES:
        path = os.path.join(DATADIR,category)
        img_array = cv2.imread(os.path.join(path,img))
        break
    return img_array

In [11]:
def get_object(data_in,rel_in):
    ob1 = data[data_in].get('relationships',)[rel_in].get('objects','')[0]
    ob2 = data[data_in].get('relationships',)[rel_in].get('objects','')[1]
    return ob1,ob2

In [12]:
#bb1 = data[4].get('objects','')[0].get('bbox','')
#bb2 = data[4].get('objects','')[3].get('bbox','').get('y','')
def get_coordinate(data_in,ob1,ob2):
    y1 = data[data_in].get('objects','')[ob1].get('bbox','').get('y','')
    x1 = data[data_in].get('objects','')[ob1].get('bbox','').get('x','')
    w1 = data[data_in].get('objects','')[ob1].get('bbox','').get('w','')
    h1 = data[data_in].get('objects','')[ob1].get('bbox','').get('h','')

    y2 = data[data_in].get('objects','')[ob2].get('bbox','').get('y','')
    x2 = data[data_in].get('objects','')[ob2].get('bbox','').get('x','')
    w2 = data[data_in].get('objects','')[ob2].get('bbox','').get('w','')
    h2 = data[data_in].get('objects','')[ob2].get('bbox','').get('h','')
    return x1,y1,w1,h1,x2,y2,w2,h2

In [14]:
#import matplotlib.pyplot as plt
#import matplotlib.patches as patches

# Create figure and axes
#fig,ax = plt.subplots(1)
# Display the image
#ax.imshow(img_array)

#Rect = []
#Rect.append(patches.Rectangle((x1,y1),w1,h1,linewidth=1,edgecolor='r',facecolor='none'))
#Rect.append(patches.Rectangle((x2,y2),w2,h2,linewidth=1,edgecolor='r',facecolor='none'))
#for i in range(len(Rect)):
#ax.add_patch(Rect[0])
#ax.add_patch(Rect[1])
#plt.show()

In [15]:
def UnionBox(x1,x2,y1,y2,h1,h2,w1,w2):
    ub_x = 0
    ub_y = 0
    ub_w = 0
    ub_h = 0

    if x1 > x2:
        ub_x = x2
    else:
        ub_x = x1

    if y1 > y2:
        ub_y = y2
    else:
        ub_y = y1

    if x1+w1 > x2+w2:
        ub_w = x1+w1-ub_x
    else:
        ub_w = x2+w2-ub_x

    if y1+h1 > y2+h2:
        ub_h = y1+h1-ub_y
    else:
        ub_h = y2+h2-ub_y
        
    return ub_x,ub_y,ub_w,ub_h

#ub_x,ub_y,ub_w,ub_h = UnionBox(x1,x2,y1,y2,h1,h2,w1,w2)
# Create figure and axes
#fig,ax = plt.subplots(1)
# Display the image
#ax.imshow(img_array)
#Rect.append(patches.Rectangle((ub_x,ub_y),ub_w,ub_h,linewidth=1,edgecolor='g',facecolor='none'))

#for i in range(len(Rect)):
#    ax.add_patch(Rect[i])
    
#plt.show()

In [16]:
def crop_image(img_arr,ub_y,ub_x,ub_w,ub_h):
    #crop the image with the union box and return it
    crop_img = img_arr[int(ub_y):int(ub_y+ub_h), int(ub_x):int(ub_x+ub_w)].copy()
    return crop_img

In [17]:
def write_to_file(data_in,rel_in,crop_img):
    
    ob1_name = data[data_in].get('relationships',)[rel_in].get('text','')[0]
    rel = data[data_in].get('relationships',)[rel_in].get('text','')[1]
    ob2_name = data[data_in].get('relationships',)[rel_in].get('text','')[2]
    
    #ob1_name = data[4].get('relationships',)[4].get('text','')[0]
    #rel = data[4].get('relationships',)[4].get('text','')[1]
    #ob2_name = data[4].get('relationships',)[4].get('text','')[2]
    image_name = data[data_in].get('filename',)[:-4]+"_"+ob1_name+"_"+rel+"_"+ob2_name+"0"+".jpg"
    name = ob1_name+"_"+rel+"_"+ob2_name
    dirName = "new_data/test/"+name
    
    try:
        # Create target Directory
        os.mkdir(dirName)
        #print("Directory " , dirName ,  " Created ") 
    except FileExistsError:
            pass
        
    filename = dirName+"/"+image_name
    exists = False
    i = int(filename[len(filename)-5:-4])
    while exists != True:
        if (os.path.isfile(filename) != True):
            cv2.imwrite(filename,crop_img)
            exists = True
        else:
            i += 1
            filename = filename[:-5]+str(i)+".jpg"

In [18]:
#write_to_file(4,4,crop_img)

In [20]:
for i in range(len(data)):
    img_array = get_image(i)
    for j in range(len(data[i].get('relationships',))):
        obj1 , obj2 = get_object(i,j)
        x1,y1,w1,h1,x2,y2,w2,h2 = get_coordinate(i,obj1,obj2)
        ub_x,ub_y,ub_w,ub_h = UnionBox(x1,x2,y1,y2,h1,h2,w1,w2)
        crop_img = crop_image(img_array,ub_y,ub_x,ub_w,ub_h)
        write_to_file(i,j,crop_img)